In [1]:
import pandas as pd
import numpy as np

In [11]:
from urllib.request import urlopen
from json import loads
url = "https://educationdata.urban.org/api/v1/schools/crdc/harassment-or-bullying/2017/race/sex/"
response = urlopen(url)
data = loads(response.read())
df = pd.DataFrame(data['results'], 
                  columns=['year', 'fips', 'race', 'sex', 'students_disc_harass_race', 'students_report_harass_race'])

In [12]:
# 2011, 2013, 2015, 2017
# cols = ['crdc_id', 'year', 'fips', 'ncessch', 'leaid', 'race', 'sex',
#        'disability', 'lep', 'students_disc_harass_dis',
#        'students_disc_harass_race', 'students_disc_harass_sex',
#        'students_report_harass_dis', 'students_report_harass_race',
#        'students_report_harass_sex']

In [13]:
df.head()

,year,fips,race,sex,students_disc_harass_race,students_report_harass_race
0,2017,1,1,1,0,0
1,2017,1,2,1,0,0
2,2017,1,3,1,0,0
3,2017,1,4,1,0,0
4,2017,1,5,1,0,0


In [14]:
df = df.rename(columns={
    "students_disc_harass_race": "num_disciplined", 
    "students_report_harass_race": "num_harassed"})
df['race'].unique()

array([ 1,  2,  3,  4,  5,  6,  7, 99])

In [15]:
df = df[df['race'].isin([i for i in range(1, 8)])]
df = df[(df['num_disciplined'] >= 0) & (df['num_harassed'] >= 0)]

In [16]:
races = {1:'White', 2:'Black', 3:'Hispanic', 4:'Asian', 5:'American Indian or Alaska Native', 
         6:'Native Hawaiian or other Pacific Islander', 7:'Two or more races'}

df = df.replace({"race": races})
df

,year,fips,race,sex,num_disciplined,num_harassed
0,2017,1,White,1,0,0
1,2017,1,Black,1,0,0
2,2017,1,Hispanic,1,0,0
3,2017,1,Asian,1,0,0
4,2017,1,American Indian or Alaska Native,1,0,0
...,...,...,...,...,...,...
9994,2017,1,Hispanic,2,0,0
9995,2017,1,Asian,2,0,0
9996,2017,1,American Indian or Alaska Native,2,0,0
9997,2017,1,Native Hawaiian or other Pacific Islander,2,0,0


In [17]:
final_df = df.groupby('race')[['num_disciplined', 'num_harassed']].sum().reset_index()
final_df

,race,num_disciplined,num_harassed
0,American Indian or Alaska Native,2,0
1,Asian,10,12
2,Black,94,148
3,Hispanic,30,38
4,Native Hawaiian or other Pacific Islander,4,0
5,Two or more races,4,4
6,White,100,72


In [18]:
df.to_csv('final_harass_bully.csv', index=False)